# 01. Plotlyインタラクティブ可視化

Plotlyを使用したインタラクティブな音響データ可視化を学びます。

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

## 1. インタラクティブ波形表示

In [ ]:
# インタラクティブな音響データ可視化
def create_interactive_waveform():
    sample_rate = 8000
    duration = 3.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 複数の音響要素を組み合わせた信号
    # 1. 基本信号（チャープ）
    chirp = signal.chirp(t, 200, duration, 800, method='linear')
    
    # 2. 倍音を持つトーン
    tone_freq = 440
    tone = (np.sin(2 * np.pi * tone_freq * t) + 
           0.5 * np.sin(2 * np.pi * tone_freq * 2 * t) +
           0.3 * np.sin(2 * np.pi * tone_freq * 3 * t))
    
    # 3. AM変調
    am_signal = tone * (1 + 0.5 * np.sin(2 * np.pi * 5 * t))
    
    # 4. ノイズ
    np.random.seed(42)
    noise = 0.1 * np.random.normal(0, 1, len(t))
    
    # エンベロープ
    envelope = np.exp(-t * 0.8)
    
    # 最終信号
    combined_signal = (chirp * 0.3 + am_signal * 0.4 + noise) * envelope
    
    return t, {
        'Combined': combined_signal,
        'Chirp': chirp * envelope * 0.3,
        'AM Tone': am_signal * envelope * 0.4,
        'Noise': noise * envelope
    }

t, signals = create_interactive_waveform()

# インタラクティブな波形プロット
fig = go.Figure()

colors = ['blue', 'red', 'green', 'orange']
for i, (name, signal) in enumerate(signals.items()):
    fig.add_trace(go.Scatter(
        x=t,
        y=signal,
        mode='lines',
        name=name,
        line=dict(color=colors[i % len(colors)]),
        visible=(name == 'Combined')  # 最初は合成信号のみ表示
    ))

# ボタンでの表示切替
buttons = []
for i, name in enumerate(signals.keys()):
    visibility = [False] * len(signals)
    visibility[i] = True
    
    buttons.append(dict(
        label=name,
        method='update',
        args=[{'visible': visibility}]
    ))

# 全て表示ボタン
buttons.append(dict(
    label='All',
    method='update',
    args=[{'visible': [True] * len(signals)}]
))

fig.update_layout(
    title='インタラクティブ波形表示',
    xaxis_title='時間 (秒)',
    yaxis_title='振幅',
    updatemenus=[dict(
        type='buttons',
        direction='left',
        buttons=buttons,
        pad={'r': 10, 't': 10},
        showactive=True,
        x=0.01,
        xanchor='left',
        y=1.15,
        yanchor='top'
    )],
    height=500
)

fig.show()

print("インタラクティブ機能:")
print("- ボタンクリックで信号成分を切り替え")
print("- ズーム・パン機能")
print("- ホバーで詳細値表示")
print("- レンジセレクター機能")

## 2. 3Dスペクトログラム

In [ ]:
# 3Dスペクトログラムの作成
sample_rate = 4000
duration = 4.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# 時間変化する複雑な信号
def create_dynamic_signal():
    signal_parts = []
    
    # セクション1: 上昇スイープ
    t1 = t[t < 1]
    sweep1 = signal.chirp(t1, 100, 1, 600, method='linear')
    signal_parts.append(sweep1)
    
    # セクション2: 複数トーン
    t2 = t[(t >= 1) & (t < 2)] - 1
    tones = (np.sin(2 * np.pi * 200 * t2) + 
            np.sin(2 * np.pi * 400 * t2) + 
            np.sin(2 * np.pi * 800 * t2))
    signal_parts.append(tones)
    
    # セクション3: FM変調
    t3 = t[(t >= 2) & (t < 3)] - 2
    carrier = 300
    modulator = 10
    fm = np.sin(2 * np.pi * carrier * t3 + 5 * np.sin(2 * np.pi * modulator * t3))
    signal_parts.append(fm)
    
    # セクション4: 下降スイープ
    t4 = t[t >= 3] - 3
    sweep2 = signal.chirp(t4, 800, 1, 150, method='linear')
    signal_parts.append(sweep2)
    
    return np.concatenate(signal_parts)

complex_signal = create_dynamic_signal()

# スペクトログラム計算
f, t_spec, Sxx = signal.spectrogram(complex_signal, sample_rate, 
                                   nperseg=256, noverlap=128)

# dB変換
Sxx_db = 10 * np.log10(Sxx + 1e-10)

# 3Dサーフェスプロット
fig_3d = go.Figure(data=[go.Surface(
    x=t_spec,
    y=f,
    z=Sxx_db,
    colorscale='Viridis',
    colorbar=dict(title='振幅 (dB)'),
    hovertemplate='時間: %{x:.2f}s<br>周波数: %{y:.0f}Hz<br>振幅: %{z:.1f}dB<extra></extra>'
)])

fig_3d.update_layout(
    title='3Dスペクトログラム',
    scene=dict(
        xaxis_title='時間 (秒)',
        yaxis_title='周波数 (Hz)',
        zaxis_title='振幅 (dB)',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    height=600
)

fig_3d.show()

# 2D等高線プロット（比較用）
fig_contour = go.Figure(data=go.Contour(
    x=t_spec,
    y=f,
    z=Sxx_db,
    colorscale='Viridis',
    colorbar=dict(title='振幅 (dB)'),
    hovertemplate='時間: %{x:.2f}s<br>周波数: %{y:.0f}Hz<br>振幅: %{z:.1f}dB<extra></extra>'
))

fig_contour.update_layout(
    title='2D等高線スペクトログラム',
    xaxis_title='時間 (秒)',
    yaxis_title='周波数 (Hz)',
    height=500
)

fig_contour.show()

print("3D可視化の利点:")
print("- 振幅の高さを直感的に把握")
print("- 回転・ズームによる多角的観察")
print("- インタラクティブなデータ探索")

## 3. 複合ダッシュボード

In [ ]:
# 音響分析ダッシュボード
def create_audio_dashboard():
    # テスト信号（楽器の音を模擬）
    sample_rate = 8000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 基音とハーモニクス
    fundamental = 220  # A3
    harmonics = [1, 2, 3, 4, 5]
    harmonic_amps = [1.0, 0.5, 0.3, 0.2, 0.1]
    
    instrument_signal = np.zeros_like(t)
    for harmonic, amp in zip(harmonics, harmonic_amps):
        freq = fundamental * harmonic
        instrument_signal += amp * np.sin(2 * np.pi * freq * t)
    
    # エンベロープ
    envelope = np.exp(-t * 1.5)
    instrument_signal *= envelope
    
    # FFT
    fft_result = fft(instrument_signal)
    freqs = fftfreq(len(instrument_signal), 1/sample_rate)
    positive_mask = freqs >= 0
    
    magnitude = np.abs(fft_result[positive_mask])
    phase = np.angle(fft_result[positive_mask])
    
    # スペクトログラム
    f_spec, t_spec, Sxx = signal.spectrogram(instrument_signal, sample_rate,
                                            nperseg=256, noverlap=128)
    
    return {
        'time': t,
        'signal': instrument_signal,
        'envelope': envelope,
        'frequencies': freqs[positive_mask],
        'magnitude': magnitude,
        'phase': phase,
        'spec_freq': f_spec,
        'spec_time': t_spec,
        'spectrogram': Sxx
    }

data = create_audio_dashboard()

# サブプロット作成
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('時間波形', 'エンベロープ', '振幅スペクトル', '位相スペクトル', 'スペクトログラム', '統計情報'),
    specs=[[{}, {}],
           [{}, {}],
           [{'colspan': 2}, None]],
    vertical_spacing=0.08,
    horizontal_spacing=0.05
)

# 1. 時間波形
fig.add_trace(
    go.Scatter(x=data['time'], y=data['signal'], 
              mode='lines', name='信号', line=dict(color='blue')),
    row=1, col=1
)

# 2. エンベロープ
fig.add_trace(
    go.Scatter(x=data['time'], y=data['envelope'], 
              mode='lines', name='エンベロープ', line=dict(color='red')),
    row=1, col=2
)

# 3. 振幅スペクトル
fig.add_trace(
    go.Scatter(x=data['frequencies'], y=20*np.log10(data['magnitude'] + 1e-10),
              mode='lines', name='振幅', line=dict(color='green')),
    row=2, col=1
)

# 4. 位相スペクトル
fig.add_trace(
    go.Scatter(x=data['frequencies'], y=data['phase'],
              mode='lines', name='位相', line=dict(color='purple')),
    row=2, col=2
)

# 5. スペクトログラム
fig.add_trace(
    go.Heatmap(x=data['spec_time'], y=data['spec_freq'], 
              z=10*np.log10(data['spectrogram'] + 1e-10),
              colorscale='Viridis', name='スペクトログラム',
              colorbar=dict(title='振幅 (dB)', x=1.02)),
    row=3, col=1
)

# レイアウト設定
fig.update_xaxes(title_text="時間 (秒)", row=1, col=1)
fig.update_yaxes(title_text="振幅", row=1, col=1)

fig.update_xaxes(title_text="時間 (秒)", row=1, col=2)
fig.update_yaxes(title_text="振幅", row=1, col=2)

fig.update_xaxes(title_text="周波数 (Hz)", row=2, col=1, range=[0, 2000])
fig.update_yaxes(title_text="振幅 (dB)", row=2, col=1)

fig.update_xaxes(title_text="周波数 (Hz)", row=2, col=2, range=[0, 2000])
fig.update_yaxes(title_text="位相 (rad)", row=2, col=2)

fig.update_xaxes(title_text="時間 (秒)", row=3, col=1)
fig.update_yaxes(title_text="周波数 (Hz)", row=3, col=1, range=[0, 1500])

fig.update_layout(
    title='音響分析ダッシュボード',
    height=800,
    showlegend=False
)

fig.show()

print("ダッシュボードの機能:")
print("- 複数の視点からの同時分析")
print("- 各グラフの独立したズーム")
print("- 統合されたホバー情報")
print("- レスポンシブデザイン")

## 練習問題

1. リアルタイムデータ更新機能を持つダッシュボードを作成してみましょう
2. 音響パラメータをスライダーで調整できるインタラクティブシンセサイザーを実装してみましょう
3. 3D空間での音源定位可視化システムを構築してみましょう